# RecordLinkage Linking

> Leveraging the RL library to determine approximate matching over a range of fields using various methods.

In [1]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

In [2]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

In [3]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
dfs = []
for key in sourceFiles.keys():
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)    
    dfs.append(vars()[key])
    print (key)

sports247
rivals
allConf
allAmerican
nflData
ncaa


In [4]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)

In [5]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)

In [6]:
df_sports247.index.name = '247_index'
df_rivals.index.name = 'rivals_index'

In [7]:
not_id = pandas.read_pickle("not_id.pkl")

In [8]:
not_id.reset_index(inplace=True)

In [9]:
noIDDict = not_id.to_dict('records')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
#df_sports247.index.name = '247_index'
#df_rivals.index.name = 'rivals_index'
df_sports247.reset_index(inplace=True)
df_rivals.reset_index(inplace=True)
#df_sports247.Index.rename('index', '247_index')
#sportDict = df_sports247.to_dict('records')
#j=0
#for i in sportDict:
#    if (j == 0):
#        print(i)
#    else:
#        break
#    j = j + 1

In [ ]:
## Hey Sober Andrew - this all works now, but you have to clean-up the output
## cause there are dueling column names between the two collections.  
## Look at the summarize code to figure out how to handle that

sportDict = df_sports247.to_dict('records')
rivalsDict = df_rivals.to_dict('records')

finalSet = []
#print(df_sports247.to_dict('records'))
for record in noIDDict:
    finalRecord = {}
    for recruit in sportDict:
        #print(recruit)
        if (recruit['247_index'] == record['247_index']):
            
            for key,value in recruit.items():
                finalRecord[key] = value
            finalRecord['confidence'] = record['sum']
            break
    for recruit in rivalsDict:
        #print(recruit)
        try:
            if (recruit['rivals_index'] == record['rivals_index']):
                for key,value in recruit.items():
                    finalRecord['rivals' + key] = value
                break
        except:
            print(recruit)
            print(record)
    finalSet.append(finalRecord)
print(finalSet)

In [ ]:
dataReview = pandas.DataFrame(finalSet)

In [ ]:
dataReview

In [ ]:
with open("..//playersForReview.csv", "w", encoding="utf-8") as write_file:
                write_file.write(dataReview.to_csv(sep='|', na_rep='N/A', ))

In [ ]:
#df_sports247.head()
df_sports247.iloc[5618, :]

In [ ]:
test = df_rivals.iloc[2673, :]
test['year']

In [ ]:
features.iloc[2673, :]

In [ ]:
print(fuzzy_matches)

In [ ]:
ids_to_match = []
ids = fuzzy_matches['ID']
for key in ids.keys():
    i = {}
    print(key[0])
    i['247'] = key[0]
    i['rivals'] = key[1]
    ids_to_match.append(i)


In [ ]:
fuzzy_list = []
for i in ids_to_match:
    sports247_dict = pandas.DataFrame(df_sports247.iloc[i['247'], :]).to_dict()
    
    rivals_dict= pandas.DataFrame(df_rivals.iloc[i['rivals'], :]).to_dict()
    for i in rivals_dict:   
        sports247_dict.update(i)
    fuzzy_list.append(sport247_dict)

In [ ]:
print(fuzzy_list)

In [ ]:
df_rivals